In [1]:
import cv2 as cv
import numpy as np

In [2]:
def gist(img):
    hist, bins = np.histogram(img, 350)
    cdf = hist.cumsum()
    cdf = (cdf-cdf[0])*255/(cdf[-1]-1)
    cdf = cdf.astype(np.uint8)
    return cdf[img]

In [3]:
def mask(rgb):
    tmp = np.ones(rgb.shape)
    rgb = np.where(rgb.sum(axis=2) == 0, 0, 255)
    rgb = rgb[..., np.newaxis]
    rgb = (rgb * tmp).astype(np.uint8)
    return rgb

In [4]:
def check_dist(prev_p, p, r=10):
    for i in range(p.shape[0]):
        for j in range(p.shape[1]):
            if(prev_p[max(0,i-r):min(p.shape[0], i + r),max(0,j-r):min(p.shape[1], j + r)].sum() == 0):
                p[i,j] = 0
    return p

In [5]:
def fb_from_video(video_file):
    cap = cv.VideoCapture(video_file)

    # Хуета 1
    frame_space = 5
    frame_array = []
    for i in range(frame_space):
        ret, first_frame = cap.read()
        hsv = np.zeros_like(first_frame)
        mask_pred = np.zeros_like(first_frame)
        prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
        frame_array.append(prev_gray)
    hsv[..., 1] = 255
    prev_p = np.nan
    num_frame = 0
    skip=10
    
    ret, first_frame = cap.read()
    prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
    hsv = np.zeros_like(first_frame)
    hsv[..., 1] = 255

    out = cv.VideoWriter('res2.mp4', cv.VideoWriter_fourcc('m','p','4','v'), cap.get(cv.CAP_PROP_FPS),
                        (frame_array[0].shape[1], frame_array[0].shape[0]))

    while(cap.isOpened()):
        # Хуета 4
        num_frame += 1

        ret, frame = cap.read()

        if ret == True:
        
            # cv.imshow("input1", frame)
            # gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            # gray = gist(gray)

            gray = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
            gray[..., 0] = 50
            gray = cv.cvtColor(gray, cv.COLOR_HSV2BGR)
            gray = cv.cvtColor(gray, cv.COLOR_BGR2GRAY)
            #cv.imshow("input2", gray)

            # cv.imshow("input2", gray)
            flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
            hsv[..., 0] = angle * 180 / np.pi / 2
            hsv[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)

            # Хуета 2
            if (hsv[..., 2].sum() == 0):
                hsv = mask_pred
            
            bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

            # Хуета 3
            frame_array.pop(0)
            frame_array.append(gray)
            mask_pred = np.copy(hsv)

            bgr = mask(bgr)

            # Хуета 5
            if num_frame > skip:
                bgr = check_dist(prev_p, bgr)
                prev_p = bgr
            elif num_frame < skip:
                if (num_frame == 1):
                    prev_p = bgr
                else:
                    prev_p += bgr
            else:
                prev_p += prev_p
                prev_p //= skip

            #cv.imshow("dense optical flow", bgr)
            prev_gray = gray
            out.write(bgr)
        else:
            break
    
    cap.release()
    cv.destroyAllWindows()

In [6]:
fb_from_video('video/6.mp4')